In [ ]:
import subprocess
from subprocess import Popen
import pandas as pd
import numpy as np
import glob
import sys
from pathlib import Path

In [ ]:
bugs = pd.read_csv('../../subjects/data/generated/bugs-in-quantum-computing-platforms.csv', sep=',')

In [ ]:
bugs.head()

In [ ]:
from datetime import datetime

buggy_elements_df = pd.DataFrame()

length = bugs.shape[0]
j = 0

for i in range(1):
    sha1 = (bugs['fix_commit_hash'].iloc[i])
    bug_id = (bugs['bug_id'].iloc[i])
    bug_type = bugs['bug_type'].iloc[i]
    
    if (sha1 == '1c07475ad9dbc30baedddf78fddafa96112ffcdc'):
        continue
    
    repo_url = (bugs['project_repository_url'].iloc[i])
    sub_repo_url = (repo_url.split('/')[-1]).split('.')[0]
    #print(sub_repo_url)
     ### Fazer checkout para a versão buggy do bug - fixed commit~1
    
    process2 = subprocess.Popen(['git','checkout',sha1],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output, errors = process2.communicate()
    
    process3 = subprocess.Popen(['git','diff-tree','--no-commit-id','--name-only','-r',sha1],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output3, errors = process3.communicate()
    
    for filepath in output3.splitlines():
        #print('NOME DO FICHEIRO')
        
        filename = filepath.split('/')[-1]
        #print(filename)
        path = filepath.rsplit('/', 1)[0]
        #print(path)
        
        if(filepath.endswith('.py')):
            
            print(filename)
            print(filepath)
            
            num_lines = sum(1 for _ in open( r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url) + '\\' + filepath))
            #print(num_lines)
            
            try:
            
                for j in range(num_lines):
                    process4 = subprocess.Popen(['git','log','--date=unix','-L', str(j) + ',' + str(j) + ':' + filename],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url) + '\\' + path, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
                    output4, errors = process4.communicate()

                    commits = []
                    authors = []
                    dates = []

                    for linha in output4.splitlines():

                        #print(linha)

                        if linha.startswith('commit'):

                            commit = linha.split("commit")[1].strip()
                            #print(commit)
                            commits.append(commit)

                        if linha.startswith('Author:'):

                            author = linha.split("Author:")[1].strip().split('<',1)[0]
                            #print(author)
                            authors.append(author)

                        if linha.startswith('Date:'):

                            date = linha.split("Date:")[1].strip()
                            dates.append(date)

                    row  = {'Bug ID': bug_id,'Hash': sha1,'Bug Type': bug_type ,'Repo' : str(sub_repo_url) ,'File': filename, 'Line': str(j),'Commits' : commits, 'Authors' : authors, 'Dates' : dates}
                    buggy_elements_df = buggy_elements_df.append(row, ignore_index = True)
                    
            except:
                print("Invalid Directory")



In [ ]:
buggy_elements_df.head()

In [ ]:
buggy_elements_df.shape

In [ ]:
buggy_elements_df['Fix commits'] = ''

In [ ]:
import re
length = buggy_elements_df.shape[0]

for i in range(length):
    
    commits = (buggy_elements_df.loc[i,['Commits']])
    fix_commits = []
    for commit in commits:
        
        if not commit:
            print("List is empty")
            
        else:
            
            ## search for message
            
            process = subprocess.Popen(['git','log','--format=%B','-n','1',str(commit[0])],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(sub_repo_url), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
            output, errors = process.communicate()
            
            for output_line in output.splitlines():
                
                pattern = re.search("fix(e[ds])?|bugs?|defects?|patch|corrigidos?|close([sd])?|resolve([sd])?", output_line, re.I)
                
                if pattern is not None:
                
                    #print(pattern)
                    fix_commits.append(str(commit[0]))
                    break

    buggy_elements_df.at[i,'Fix commits'] = fix_commits
                
                
    
   
        

In [ ]:
buggy_elements_df.head()

## Converting hash commits to dates

In [ ]:
buggy_elements_df['Bug Probability'] = ''

In [ ]:
def date_normalized(x,y):
    
    dates = []
    for commit in x:
        
        process = subprocess.Popen(['git','show','--no-patch','--no-notes','--pretty=' + r'%cd','--format=' + r"%at",commit],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + y, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
        output, errors = process.communicate()
        
        print(str(output.splitlines()[0].rstrip()))
        dates.append(int(output.splitlines()[0].rstrip()))
       
        
    return dates
            
    
    

In [ ]:
import numpy as np

def NormalizeData(data):
    
    data =  np.array(data)
    
    if len(data) == 1:
        
        return [1]
    
    else:
        
        return (data - np.min(data)) / (np.max(data) - np.min(data))


In [ ]:
import math

def TWRFunction(normal_data):
    
    twr = 0
    for i in normal_data:
        
        twr = twr + 1/1 + math.e**(-12*i + 12)
    
    defect_probability = 1 -math.e**-twr
    
    return round(defect_probability,2)
        

In [ ]:
length = buggy_elements_df.shape[0]

for i in range(length):
    
    #print(buggy_elements_df['Commits'].iloc[i])
    #print(buggy_elements_df['Repo'].iloc[i])
    
    if not buggy_elements_df['Fix commits'].iloc[i]:
        
        try:
            buggy_elements_df.at[i,'Bug Probability'] = 0.3 * TWRFunction(NormalizeData(date_normalized(buggy_elements_df['Commits'].iloc[i],buggy_elements_df['Repo'].iloc[i])))
        
        except ValueError:
             print('Commit List is Empty')
            
    else:    
    
        try:
            buggy_elements_df.at[i,'Bug Probability'] = 0.7 * TWRFunction(NormalizeData(date_normalized(buggy_elements_df['Fix commits'].iloc[i],buggy_elements_df['Repo'].iloc[i]))) + 0.3 * TWRFunction(NormalizeData(date_normalized(buggy_elements_df['Commits'].iloc[i],buggy_elements_df['Repo'].iloc[i])))

        except ValueError:
            print('Commit List is Empty')

In [ ]:
buggy_elements_df.to_csv('..\data\RQ3.0_Data.csv', index=False, encoding="utf8" )

In [ ]:
buggy_elements_df['Bug Probability'].unique()

In [ ]:
#final_df = buggy_elements_df[buggy_elements_df['Fix commits'].map(lambda d: len(d)) > 0]
#git show --no-patch --no-notes --pretty='%cd' --format="%at" ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666